# MDF data analysis

In [18]:
from load_data import train, test, BIG, TRAIN, TEST, SOURCE
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from visualization_helper import display_groups, display_unique_count
from preprocessing_helper import get_unique_noconst, split_series_values, get_unique_col_values

In [3]:
print BIG.shape, BIG.columns.values
print train.shape, train.columns.values
print test.shape, test.columns.values


(12235, 43) [u'agrement col' u'date amm annee' u'date declar annee' u'etat commerc'
 u'forme pharma' u'id' u'libelle' u'libelle_ampoule' u'libelle_capsule'
 u'libelle_comprime' u'libelle_film' u'libelle_flacon' u'libelle_gelule'
 u'libelle_pilulier' u'libelle_plaquette' u'libelle_poche'
 u'libelle_sachet' u'libelle_seringue' u'libelle_stylo' u'libelle_tube'
 u'nb_ampoule' u'nb_capsule' u'nb_comprime' u'nb_film' u'nb_flacon'
 u'nb_gelule' u'nb_ml' u'nb_pilulier' u'nb_plaquette' u'nb_poche'
 u'nb_sachet' u'nb_seringue' u'nb_stylo' u'nb_tube' u'prix' 'source'
 u'statut' u'statut admin' u'substances' u'titulaires' u'tx rembours'
 u'type proc' u'voies admin']
(8564, 42) [u'libelle' u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon'
 u'libelle_tube' u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier'
 u'libelle_sachet' u'libelle_comprime' u'libelle_gelule' u'libelle_film'
 u'libelle_poche' u'libelle_capsule' u'nb_plaquette' u'nb_ampoule'
 u'nb_flacon' u'nb_tube' u'nb_stylo' u'nb_

Remove some useless columns

In [4]:
df__noid_noprix = BIG.drop(['libelle', 'prix', 'id'], axis=1)

Check for dupicate rows and lets look at the columns with only one unique value.

In [5]:
df__noid_noprix_unique_noconst = get_unique_noconst(df__noid_noprix, verbose=True)

-- get_unique_noconst --
- Number of found duplicated rows :  1389
- Remove constant columns :  (10846, 40) -> (10846, 40)
- Dropped const columns : 


Columns description : 

In [6]:
type_groups_func = lambda df : df.columns.to_series().groupby(df.dtypes)
type_groups = type_groups_func(df__noid_noprix_unique_noconst)

In [7]:
display_groups(type_groups)


-- object --

	agrement col
	etat commerc
	forme pharma
	source
	statut
	statut admin
	substances
	titulaires
	tx rembours
	type proc
	voies admin

-- int64 --

	date amm annee
	date declar annee
	libelle_ampoule
	libelle_capsule
	libelle_comprime
	libelle_film
	libelle_flacon
	libelle_gelule
	libelle_pilulier
	libelle_plaquette
	libelle_poche
	libelle_sachet
	libelle_seringue
	libelle_stylo
	libelle_tube
	nb_ampoule
	nb_capsule
	nb_comprime
	nb_film
	nb_flacon
	nb_gelule
	nb_pilulier
	nb_plaquette
	nb_poche
	nb_sachet
	nb_seringue
	nb_stylo
	nb_tube

-- float64 --

	nb_ml


Identify and separate the numeric and non numeric rows.
Report NaN values on numerical columns

In [9]:
num_cols, nonnum_cols = get_numeric_nonnumeric_cols(df__noid_noprix_unique_noconst)
        
nb_isnan = np.sum(df__noid_noprix_unique_noconst[num_cols].apply(np.isnan))

In [10]:
print nb_isnan

libelle_plaquette    0
libelle_ampoule      0
libelle_flacon       0
libelle_tube         0
libelle_stylo        0
libelle_seringue     0
libelle_pilulier     0
libelle_sachet       0
libelle_comprime     0
libelle_gelule       0
libelle_film         0
libelle_poche        0
libelle_capsule      0
nb_plaquette         0
nb_ampoule           0
nb_flacon            0
nb_tube              0
nb_stylo             0
nb_seringue          0
nb_pilulier          0
nb_sachet            0
nb_comprime          0
nb_gelule            0
nb_film              0
nb_poche             0
nb_capsule           0
date declar annee    0
date amm annee       0
nb_ml                0
dtype: int64


Identify non numeric rows.

In [11]:
df__noid_noprix_unique_noconst[nonnum_cols].tail(5)

,agrement col,etat commerc,forme pharma,source,statut,statut admin,substances,titulaires,tx rembours,type proc,voies admin
12230,oui,Déclaration de commercialisation,solution injectable,test,Présentation active,Autorisation active,KÉTOPROFÈNE,PHARMY II,30%,Procédure nationale,intramusculaire
12231,oui,Déclaration de commercialisation,solution injectable,test,Présentation active,Autorisation active,ACIDE GADOTÉRIQUE,GUERBET,65%,Procédure nationale,intra-articulaire
12232,oui,Déclaration de commercialisation,comprimé gastro-résistant(e),test,Présentation active,Autorisation active,"PANTOPRAZOLE SODIQUE SESQUIHYDRATÉ, PANTOPRAZOLE",RANBAXY PHARMACIE GENERIQUES,65%,Procédure décentralisée,orale
12233,oui,Déclaration de commercialisation,solution pour perfusion,test,Présentation active,Autorisation active,"ACIDE ZOLÉDRONIQUE ANHYDRE, ACIDE ZOLÉDRONIQUE...",BIOGARAN,65%,Procédure nationale,intraveineuse
12234,oui,Déclaration de commercialisation,capsule molle,test,Présentation active,Autorisation active,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",PHARMA DEVELOPPEMENT,65%,Procédure nationale,orale


In [19]:
train__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TRAIN]
train_unique_col_values = display_unique_col_values(train__noid_noprix_unique_noconst[nonnum_cols])
train_unique_col_values

,0,1,2,3,4,5,6,7,8,9,...,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043
0,agrement col,2,oui,non,,,,,,,...,,,,,,,,,,
1,etat commerc,4,Déclaration de commercialisation,Déclaration d'arrêt de commercialisation,Arrêt de commercialisation (le médicament n'a ...,Déclaration de suspension de commercialisation,,,,,...,,,,,,,,,,
2,forme pharma,218,comprimé pelliculé,comprimé à croquer,gélule à libération prolongée,granulés à libération prolongée,comprimé,capsule molle,comprimé sécable,dispositif,...,,,,,,,,,,
3,source,2,train,test,,,,,,,...,,,,,,,,,,
4,statut,2,Présentation active,Présentation abrogée,,,,,,,...,,,,,,,,,,
5,statut admin,5,Autorisation active,Autorisation abrogée,Autorisation archivée,Autorisation retirée,Autorisation suspendue,,,,...,,,,,,,,,,
6,substances,2042,DÉSOGESTREL,"MONTÉLUKAST ACIDE, MONTÉLUKAST SODIQUE",CHLORHYDRATE DE DILTIAZEM,"ACIDE VALPROÏQUE, VALPROATE DE SODIUM",LÉTROZOLE,FLUTAMIDE,BEXAROTÈNE,GLIMÉPIRIDE,...,TRIMÉBUTINE (MALÉATE DE),"SODIUM (DIHYDROGÉNOPHOSPHATE DE) DIHYDRATÉ, HY...","TRAMADOL, CHLORHYDRATE DE TRAMADOL","ISONIAZIDE, RIFAMPICINE","RILPIVIRINE, FUMARATE DE TÉNOFOVIR DISOPROXIL,...","BROMURE D'IPRATROPIUM ANHYDRE, BROMURE D'IPRAT...",CALCIUM (FLUORURE DE),BÉCLOMÉTHASONE (DIPROPIONATE DE),"RANITIDINE, CHLORHYDRATE DE RANITIDINE","ACÉTATE D'OCTRÉOTIDE, OCTRÉOTIDE"
7,titulaires,353,MAJORELLE LUXEMBOURG SOPARFI (LUXEMBOURG),TEVA SANTE,MYLAN SAS,SANOFI AVENTIS FRANCE,EG LABO - LABORATOIRES EUROGENERICS,EISAI (ROYAUME-UNI),SANDOZ,ARROW GENERIQUES,...,,,,,,,,,,
8,tx rembours,5,65%,100%,15%,30%,0%,,,,...,,,,,,,,,,
9,type proc,5,Procédure décentralisée,Procédure de reconnaissance mutuelle,Procédure nationale,Procédure centralisée,Autorisation d'importation parallèle,,,,...,,,,,,,,,,


In [20]:
test__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TEST]
test_unique_col_values = display_unique_col_values(test__noid_noprix_unique_noconst[nonnum_cols])
test_unique_col_values

,0,1,2,3,4,5,6,7,8,9,...,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143
0,agrement col,2,non,oui,,,,,,,...,,,,,,,,,,
1,etat commerc,3,Déclaration d'arrêt de commercialisation,Déclaration de commercialisation,Arrêt de commercialisation (le médicament n'a ...,,,,,,...,,,,,,,,,,
2,forme pharma,159,comprimé gastro-résistant(e),capsule,comprimé,suspension injectable,émulsion pour perfusion,comprimé pelliculé sécable,solution pour perfusion,comprimé pelliculé,...,,,,,,,,,,
3,source,1,test,,,,,,,,...,,,,,,,,,,
4,statut,2,Présentation active,Présentation abrogée,,,,,,,...,,,,,,,,,,
5,statut admin,5,Autorisation active,Autorisation abrogée,Autorisation archivée,Autorisation retirée,Autorisation suspendue,,,,...,,,,,,,,,,
6,substances,1142,"PANTOPRAZOLE SODIQUE SESQUIHYDRATÉ, PANTOPRAZOLE","NYSTATINE, SULFATE DE POLYMYXINE B, SULFATE DE...",PRAVASTATINE SODIQUE,INSULINE HUMAINE RECOMBINANTE ISOPHANE BIPHASI...,"PHOSPHATE MONOSODIQUE DIHYDRATÉ, ARGININE, MAG...","MÉMANTINE, CHLORHYDRATE DE MÉMANTINE",CHLORURE DE SODIUM,"FUMARATE DE TÉNOFOVIR DISOPROXIL, TÉNOFOVIR DI...",...,"BROMURE D'IPRATROPIUM ANHYDRE, BROMURE D'IPRAT...","HYDROCHLOROTHIAZIDE, RAMIPRIL","PÉRINDOPRIL, PÉRINDOPRIL ARGININE, INDAPAMIDE",CALCIUM (FLUORURE DE),BÉCLOMÉTHASONE (DIPROPIONATE DE),INSULINE HUMAINE RECOMBINANTE ISOPHANE,"CEFPODOXIME PROXÉTIL, CÉFUROXIME",PRUNIER D'AFRIQUE (EXTRAIT DE),"RANITIDINE, CHLORHYDRATE DE RANITIDINE","ACÉTATE D'OCTRÉOTIDE, OCTRÉOTIDE"
7,titulaires,265,TEVA SANTE,INNOTECH INTERNATIONAL,ACTAVIS FRANCE,SANOFI AVENTIS (ALLEMAGNE),B BRAUN MELSUNGEN,BIOGARAN,FRESENIUS KABI FRANCE,GILEAD SCIENCES INTERNATIONAL (ROYAUME-UNI),...,,,,,,,,,,
8,tx rembours,4,65%,30%,15%,100%,,,,,...,,,,,,,,,,
9,type proc,5,Procédure de reconnaissance mutuelle,Procédure nationale,Procédure centralisée,Procédure décentralisée,Autorisation d'importation parallèle,,,,...,,,,,,,,,,


In [15]:
index = 10
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

208
                           Value  Count
24                 intraveineuse     34
12             intra-articulaire     17
13              intra-artérielle     16
20               intramusculaire     15
30                         orale     11
23                  intrathécale     10
36                  sous-cutanée      9
15                 intra-utérine      8
31               périarticulaire      7
35                       rectale      6
32                    péridurale      6
4                  endocanalaire      5
19              intralésionnelle      5
33                   périneurale      4
5                   endosinusale      4
9                   infiltration      4
42    voie extracorporelle autre      3
34                  périoculaire      3
41            voie buccale autre      3
8                      gingivale      3
43        voie parentérale autre      3
27                 intravésicale      2
38                 transdermique      2
28                        nasale    

In [14]:
index = 8
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

325
                                                Value  Count
301                                   TEVA (PAYS-BAS)      2
208             MERCK SHARP & DOHME (GRANDE BRETAGNE)      2
213                                       MUNDIPHARMA      1
220                                NORGINE (PAYS-BAS)      1
219                                     NORDIC PHARMA      1
218                                     NEXMED PHARMA      1
217                                            NEPALM      1
216                                         NELSONIUS      1
215                                         MYLAN SAS      1
214                                     MYLAN MEDICAL      1
0                                     ABBOTT PRODUCTS      1
221                                    NORGINE PHARMA      1
211                                  MOLTENI & ALITTI      1
210                                       MITHRIDATUM      1
209                    MERCK SHARP & DOHME (PAYS-BAS)      1
207                 

In [18]:
index = 9
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df[df['Count'] > 5]

3800
                                                  Value  Count
287                            CLAVULANATE DE POTASSIUM     33
77                             AMOXICILLINE TRIHYDRATÉE     31
463                                 HYDROCHLOROTHIAZIDE     30
1508                                HYDROCHLOROTHIAZIDE     24
10                                   ACIDE CLAVULANIQUE     24
162                                     CALCIUM ÉLÉMENT     22
69                                           AMLODIPINE     22
268                                     CHOLÉCALCIFÉROL     21
901                                    ÉTHINYLESTRADIOL     18
442                                 GLUCOSE MONOHYDRATÉ     18
706                                         PÉRINDOPRIL     18
409                                  FORME PULVÉRULENTE     18
74                                 AMOXICILLINE ANHYDRE     17
47                                              ALANINE     16
141                               BÉSILATE D'AMLOD

Encode categorical features

In [16]:
df = df__noid_noprix_unique_noconst[nonnum_cols].drop(['substances', 'voies admin'], axis=1)
out_df = pd.DataFrame(columns=df.columns, index=df.index)
for c in df.columns:
    le = LabelEncoder()    
    le.fit(df[c])
    out_df[c] = le.transform(df[c])
out_df

,statut,etat commerc,agrement col,tx rembours,forme pharma,statut admin,type proc,titulaires
0,1,2,1,4,30,1,3,194
1,1,2,1,4,49,1,2,307
2,1,2,1,4,78,1,4,218
3,1,2,1,4,74,1,4,275
4,1,1,0,1,30,1,4,307
5,1,2,1,1,7,1,4,92
6,1,2,1,1,1,1,1,93
7,1,2,1,4,7,1,2,272
8,1,1,1,4,44,1,4,32
9,1,2,0,4,30,1,4,32
